# Planes de pensiones y ALM

In [40]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [19]:
sal = pd.Series(np.array([29066, 36483,58406,35173,46354,34567]))
in1 = 1.02
eda = np.array([46, 38, 63, 30, 30, 24])
proyec = round(sal*(in1)**(65-1-eda), 2)
pj = round(.25*proyec/12, 2)
proyec

0    41513.41
1    61051.31
2    59574.12
3    68962.86
4    90885.18
5    76325.31
dtype: float64

In [56]:
def prestacion_definida(df, fecha_valoracion = '31/12/2021',  rvs = .015, rvp = .01,  h=12, margen_sol = .02,
                        tipo_int= .0022, beta=.15, edad_jubilacion = 65):
    def lx_participes():

        raw1 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/Participes_aportacion.csv'
        data1 = pd.read_csv(raw1).rename(columns = {'Antoni': 'Joan'})
        raw2 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/approximation_planex.csv'
        data2 = pd.read_csv(raw2).rename(columns = {'Antoni': 'Joan'})
        del data1['Unnamed: 0']
        del data2['Unnamed: 0']
        return data1, data2

    data1 = lx_participes()[0]
    data2 = lx_participes()[1]
    year = int(fecha_valoracion.split('/')[2])
#     l1, l2 = [], []
#     l3, l4 = [], []
#     l5 = []
#     participe = int(input('Ingresar el nùmero de participes del plan: '))
#     for i in range(0, participe):
        
#         p1 = str(input(f'Poner el nombre del participe del Plan numero {i+1}: '))
#         p2 = str(input(f'Ingresar la fecha de nacimiento {p1}: '))
#         p3 = str(input(f'Ingresar la fecha de alta en la empresa de {p1}: '))
#         p4 = float(input(f'Ingresar el salario pensionable 2021 de {p1}: '))
#         p5 = str(input(f'Ingresar el Sexo de {p1}: '))
#         print('='*55)
#         l1.append(p1)
#         l2.append(p2)
#         l3.append(p3)
#         l4.append(p4)
#         l5.append(p5)

    array1 = []
    for i in range(0, len(df['Participe'])):
        array1.append(int(df['Nacimiento'][i].split('/')[2]))

    array2 = []
    for i in range(0, len(df['Participe'])):
        array2.append(int(df['Alta'][i].split('/')[2]))
    an1 = [46, 38, 63, 30, 30, 24] ## edad actuarial  # AAA poner manualmente poner esto
    an2 = [25, 37, 29, 26, 20, 23] ## edad entradad actuarial  # AAA poner manualmente esto
    edad = [x-y for(x, y) in zip(array2, array1)] ## edad actuarial 2
    r1 = 1+rvs  ## Revalorizaciòn salarios
    r2 = 1+rvp  ## Revalorizaciòn pensiones 
    interes = (1+tipo_int)  ##interés
    t1 = np.arange(0, (120-edad_jubilacion)*h)  ## mensilidad
    interes2 = interes**(1/h)  ## fraccionamiento 1/h
    interes3 = interes2**-t1  ## intereses mensuales
    def variation_capital():
        j = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/variation6.csv'
        ok = pd.read_csv(j)
        k = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/px_mens_participes.csv'
        ok1 = pd.read_csv(k)
        return ok, ok1
    hello = variation_capital()
    ok, ok1 = hello[0], hello[1]
#     df = pd.DataFrame({'Participe':pd.Series(l1), 'Sexo': pd.Series(l5),'Nacimiento':pd.Series(l2), 
#                        'Alta':pd.Series(l3), 'Salario 2021':pd.Series(l4), 
#                       'Edad actuarial':pd.Series(an1), 'Edad entrada':pd.Series(an2)})

    e1 = round(((pd.to_datetime(datos['Alta']) - pd.to_datetime(datos['Nacimiento']))/365.25)/np.timedelta64(1, 'D')) ## Edad E
    fec_val= []
    for i in range(0,6):
        fec_val.append(fecha_valoracion)
    fec_val = pd.Series(fec_val)
    act_edad = round(((pd.to_datetime(fec_val) - pd.to_datetime(datos['Nacimiento']))/365.25)/np.timedelta64(1, 'D')) ## Edad A
    df = df.join(pd.Series(act_edad, name='Edad actuarial'))
    df = df.join(pd.Series(e1, name='Edad entrada'))
    jubi2 = pd.to_numeric(df['Nacimiento'].str.split('/', expand=True)[2])+edad_jubilacion
    df = df.join(pd.Series(jubi2, name='Jubilacion'))
    proyec = round(df['Salario 2021']*(r1)**(edad_jubilacion-1-df['Edad actuarial']), 2)
    pj = round(beta*proyec/12, 2)
    df['Proyectado'] = proyec
    df['Proyectado'][2] = datos['Salario 2021'][2]
    
    jubi3 = pd.to_numeric(df['Alta'].str.split('/', expand=True)[2])
    df['PJ corriente'] = pj
    df['PJ corriente'][2] = beta*df['Proyectado'][2]/12
    joan1 = ok['Joan_var']*ok1['Joan']
    pere1 = ok['Pere_var']*ok1['Pere']
    marta1 = ok['Marta_var']*ok1['Marta']
    claudia1 = ok['Claudia_var']*ok1['Claudia']
    marc1 = ok['Marc_var']*ok1['Marc']
    maria1 = ok['Maria_var']*ok1['Maria']
    new_df = pd.DataFrame({'Joan_VAA':joan1*interes3, 'Pere_VAA':pere1*interes3, 'Marta_VAA':marta1*interes3,
                           'Claudia_VAA':claudia1*interes3, 'Marc_VAA':marc1*interes3, 'Maria_VAA':maria1*interes3})
    VAAs = pd.Series(new_df.sum().round(2).to_list(), name='VAAj')
    
    ## A la fecha actual 2021
    VAA21_joan = round(VAAs[0]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][0]))*(data1['Joan'][edad_jubilacion]/data1['Joan'][df['Edad actuarial'][0]]), 2)
    VAA21_pere = round(VAAs[1]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][1]))*(data1['Pere'][65]/data1['Pere'][40]), 2)
    VAA21_marta = round(VAAs[2]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][2]))*(data1['Marta'][65]/data1['Marta'][65]), 2)
    VAA21_claudia = round(VAAs[3]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][3]))*(data1['Claudia'][65]/data1['Claudia'][32]), 2)
    VAA21_marc = round(VAAs[4]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][4]))*(data1['Marc'][65]/data1['Marc'][32]), 2)
    VAA21_maria = round(VAAs[5]*((1+tipo_int)**-(edad_jubilacion-df['Edad actuarial'][5]))*(data1['Maria'][65]/data1['Maria'][26]), 2)
    vaas21 = pd.Series([VAA21_joan, VAA21_pere, VAA21_marta, VAA21_claudia, VAA21_marc, VAA21_maria])
    
    ## A la fecha de alta en la empresa
    VAAp_joan = round(VAAs[0]*((1+tipo_int)**-(jubi2[0]-jubi3[0]+1))*(data1['Joan'][65]/data1['Joan'][25]), 2)
    VAAp_pere = round(VAAs[1]*((1+tipo_int)**-(jubi2[1]-jubi3[1]+1))*(data1['Pere'][65]/data1['Pere'][37]), 2)
    VAAp_marta = round(VAAs[2]*((1+tipo_int)**-(jubi2[2]-jubi3[2]+1))*(data1['Marta'][65]/data1['Marta'][29]), 2)
    VAAp_claudia = round(VAAs[3]*((1+tipo_int)**-(jubi2[3]-jubi3[3]+1))*(data1['Claudia'][65]/data1['Claudia'][26]), 2)
    VAAp_marc = round(VAAs[4]*((1+tipo_int)**-(jubi2[4]-jubi3[4]+1))*(data1['Marc'][65]/data1['Marc'][20]), 2)
    VAAp_maria = round(VAAs[5]*((1+tipo_int)**-(jubi2[5]-jubi3[5]+1))*(data1['Maria'][65]/data1['Maria'][23]), 2)
    vaas_p = pd.Series([VAAp_joan, VAAp_pere, VAAp_marta, VAAp_claudia, VAAp_marc, VAAp_maria])
    
    ## Coste normal Credito Unitario
    jo1 = VAAs[0]*(1+tipo_int)**-(edad_jubilacion-np.arange(df['Edad entrada'][0], df['Edad actuarial'][0]+1))*(data1['Joan'][65]/data1['Joan'][np.arange(df['Edad entrada'][0], df['Edad actuarial'][0]+1)])
    jo2 = VAAs[1]*(1+tipo_int)**-(edad_jubilacion-np.arange(df['Edad entrada'][1], df['Edad actuarial'][1]+1))*(data1['Pere'][65]/data1['Pere'][np.arange(df['Edad entrada'][1], df['Edad actuarial'][1]+1)])
    jo3 = VAAs[2]*(1+tipo_int)**-(edad_jubilacion-np.arange(df['Edad entrada'][2], df['Edad actuarial'][2]+1))*(data1['Marta'][65]/data1['Marta'][np.arange(df['Edad entrada'][2], df['Edad actuarial'][2]+1)])
    jo4 = VAAs[3]*(1+tipo_int)**-(edad_jubilacion-np.arange(df['Edad entrada'][3], df['Edad actuarial'][3]+1))*(data1['Claudia'][65]/data1['Claudia'][np.arange(df['Edad entrada'][3], df['Edad actuarial'][3]+1)])
    jo5 = VAAs[4]*(1+tipo_int)**-(edad_jubilacion-np.arange(df['Edad entrada'][4], df['Edad actuarial'][4]+1))*(data1['Marc'][65]/data1['Marc'][np.arange(df['Edad entrada'][4], df['Edad actuarial'][4]+1)])
    jo6 = VAAs[5]*(1+tipo_int)**-(edad_jubilacion-np.arange(df['Edad entrada'][5], df['Edad actuarial'][5]+1))*(data1['Maria'][65]/data1['Maria'][np.arange(df['Edad entrada'][5], df['Edad actuarial'][5]+1)])
    CN_credito = pd.Series([jo1.iloc[-1]/(edad_jubilacion-df['Edad entrada'][0]), 
                            jo2.iloc[-1]/(edad_jubilacion-df['Edad entrada'][1]),
                            jo3.iloc[-1]/(edad_jubilacion-df['Edad entrada'][2]),
                            jo4.iloc[-1]/(edad_jubilacion-df['Edad entrada'][3]), 
                            jo5.iloc[-1]/(edad_jubilacion-df['Edad entrada'][4]), 
                            jo6.iloc[-1]/(edad_jubilacion-df['Edad entrada'][5])]).round(2)
    VAASP_CU = pd.Series([  CN_credito[0]*(df['Edad actuarial'][0]-df['Edad entrada'][0]),
                            CN_credito[1]*(df['Edad actuarial'][1]-df['Edad entrada'][1]),
                            CN_credito[2]*(df['Edad actuarial'][2]-df['Edad entrada'][2]),
                            CN_credito[3]*(df['Edad actuarial'][3]-df['Edad entrada'][3]),
                            CN_credito[4]*(df['Edad actuarial'][4]-df['Edad entrada'][4]),
                            CN_credito[5]*(df['Edad actuarial'][5]-df['Edad entrada'][5])])
    ## Margen de solvencia 
    margen_CU = VAASP_CU*margen_sol
    
    ## dataframe Credito Unitario
    CU = pd.DataFrame({'VAAj':VAAs,'VAA 31/12/2021': vaas21, 'Alta Firm':vaas_p,
                       'CN 31/12/2021':CN_credito, 'VAASP 31/12/2021':VAASP_CU, 'Margen':margen_CU})
    CU.index = df['Participe'].to_list()
    
    ## Edad de entrada -Coste Normal
    tiempo1 = np.arange(0, (df['Edad actuarial'][0]-df['Edad entrada'][0]))
    jo_1 = vaas_p[0]/sum(((1+rvs)**tiempo1*(1+tipo_int)**-(tiempo1))*(data1['Joan'][np.arange(df['Edad entrada'][0],
                                                                                  df['Edad actuarial'][0])]/data1['Joan'][df['Edad entrada'][0]]))
    tiempo2 = np.arange(0, (df['Edad actuarial'][1]-df['Edad entrada'][1]))
    jo_2 = vaas_p[1]/sum(((1+rvs)**tiempo2*(1+tipo_int)**-(tiempo2))*(data1['Pere'][np.arange(df['Edad entrada'][1],
                                                                                  df['Edad actuarial'][1])]/data1['Pere'][df['Edad entrada'][1]]))
    tiempo3 = np.arange(0, (df['Edad actuarial'][2]-df['Edad entrada'][2]))
    jo_3 = vaas_p[2]/sum(((1+rvs)**tiempo3*(1+tipo_int)**-(tiempo3))*(data1['Marta'][np.arange(df['Edad entrada'][2],
                                                                                  df['Edad actuarial'][2])]/data1['Claudia'][df['Edad entrada'][2]]))
    tiempo4 = np.arange(0, (df['Edad actuarial'][3]-df['Edad entrada'][3]))
    jo_4 = vaas_p[3]/sum(((1+rvs)**tiempo4*(1+tipo_int)**-(tiempo4))*(data1['Claudia'][np.arange(df['Edad entrada'][3],
                                                                                  df['Edad actuarial'][3])]/data1['Marta'][df['Edad entrada'][3]]))
    tiempo5 = np.arange(0, (df['Edad actuarial'][4]-df['Edad entrada'][4]))
    jo_5 = vaas_p[4]/sum(((1+rvs)**tiempo5*(1+tipo_int)**-(tiempo5))*(data1['Marc'][np.arange(df['Edad entrada'][4],
                                                                                  df['Edad actuarial'][4])]/data1['Marc'][df['Edad entrada'][4]]))
    tiempo6 = np.arange(0, (df['Edad actuarial'][5]-df['Edad entrada'][5]))
    jo_6 = vaas_p[5]/sum(((1+rvs)**tiempo6*(1+tipo_int)**-(tiempo6))*(data1['Maria'][np.arange(df['Edad entrada'][5],
                                                                                  df['Edad actuarial'][5])]/data1['Maria'][df['Edad entrada'][5]]))
    CN_EE = pd.Series([jo_1,jo_2,jo_3,jo_4,jo_5,jo_6 ])
    print('='*75)
    print('Hipotesis')
    print(f'Interés técnico valoración: {tipo_int*100}%')
    print(f'Revalorizaciòn Salario: {rvs*100}%')
    print(f'Revalorización Pensión: {rvp*100}%')
    print(f'Beta Pension: {beta}')
    print('='*75)
    print(df.set_index('Participe').T)
    print('='*75)
    print('Credito Unitario')
    print(round(CU.T, 2))
    print('='*75)
    print(round(CN_EE, 2))
    #abc = ((1+rvs)**tiempo6*(1+tipo_int)**-(tiempo6))*(data1['Maria'][np.arange(df['Edad entrada'][5],
     #                                                                             df['Edad actuarial'][5])]/data1['Maria'][df['Edad entrada'][5]])
    #return abc
    #return VAASP_CU, CN_credito
    

In [32]:
datos = pd.DataFrame({'Participe':['Joan', 'Pere', 'Marta', 'Claudia', 'Marc', 'Maria'], 
                     'Nacimiento':['06/07/1973', '03/12/1981', '06/03/1957','09/11/1989','10/07/1990',
                                  '11/09/1995'],
                     'Alta':['15/02/1998','08/03/2019','01/09/1985','09/10/2015','01/10/2010',
                            '08/08/2018'],
                     'Salario 2021':[29944, 37586, 60172, 36236, 47755, 35612]})
datos

,Participe,Nacimiento,Alta,Salario 2021
0,Joan,06/07/1973,15/02/1998,29944
1,Pere,03/12/1981,08/03/2019,37586
2,Marta,06/03/1957,01/09/1985,60172
3,Claudia,09/11/1989,09/10/2015,36236
4,Marc,10/07/1990,01/10/2010,47755
5,Maria,11/09/1995,08/08/2018,35612


In [6]:
126016/sum(1.01**np.arange(0, (47-19))*1.0125**-(np.arange(0, (47-19)))*data1['Antoni'][np.arange(19, 47)]/data1['Antoni'][19])

4680.726638398424

In [42]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Participe             Joan        Pere       Marta     Claudia        Marc  \
Nacimiento      06/07/1973  03/12/1981  06/03/1957  09/11/1989  10/07/1990   
Alta            15/02/1998  08/03/2019  01/09/1985  09/10/2015  01/10/2010   
Salario 2021         29944       37586       60172       36236       47755   
Edad actuarial        49.0        41.0        65.0        32.0        31.0   
Edad entrada          25.0        38.0        28.0        26.0        19.0   
Jubilacion            2038        2046        2022        2054        2055   
Proyectado        37436.95    52935.26     60172.0    58351.71    78054.55   
PJ corriente        467.96      661.69      752.15       729.4      975.68   

Participe            Maria  
Nacimiento      11/09/1995  
Alta            08/08/2018  
Salario 2021         35612  
Edad actuarial        26.0  
Edad entrada          23.0  
Ju

In [49]:
var = np.roll(np.round(783.82*1.015**np.floor(np.arange(0,660) / 12 ), 2),-8)
var[-8:] = var[-9]
var[0] = 0
var_m3 = pd.Series(var)
var_m3

0         0.00
1       783.82
2       783.82
3       783.82
4       795.58
        ...   
655    1751.39
656    1751.39
657    1751.39
658    1751.39
659    1751.39
Length: 660, dtype: float64

In [51]:
dt1 = pd.DataFrame({'Joan_var':var_jo, 'Pere_var':var_pe, 'Marta_var':var_m1, 'Claudia_var':var_cl, 'Marc_var':var_m2,
                   'Maria_var':var_m3})
dt1.to_csv('variation6.csv')

In [28]:
var = np.roll(np.round(r2[0]*1.02**np.floor(np.arange(0,660) / 12 ), 2),-(r1[0]-1))
var[-(r1[0]-1):] = var[-(r1[0])]
var[0] = 0
var = pd.Series(var)
var

0         0.00
1       467.96
2       467.96
3       467.96
4       467.96
        ...   
655    1363.38
656    1363.38
657    1363.38
658    1363.38
659    1363.38
Length: 660, dtype: float64

In [27]:
var = np.roll(np.round(467.96*1.01**np.floor(np.arange(0,660) / 12 ), 2),-6)
var[-6:] = var[-7]
var[0] = 0
var = pd.Series(var)
var

0        0.00
1      467.96
2      467.96
3      467.96
4      467.96
        ...  
655    800.87
656    800.87
657    800.87
658    800.87
659    800.87
Length: 660, dtype: float64

In [55]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Participe             Joan        Pere       Marta     Claudia        Marc  \
Nacimiento      06/07/1973  03/12/1981  06/03/1957  09/11/1989  10/07/1990   
Alta            15/02/1998  08/03/2019  01/09/1985  09/10/2015  01/10/2010   
Salario 2021         29944       37586       60172       36236       47755   
Edad actuarial        49.0        41.0        65.0        32.0        31.0   
Edad entrada          25.0        38.0        28.0        26.0        19.0   
Jubilacion            2038        2046        2022        2054        2055   
Proyectado        37436.95    52935.26     60172.0    58351.71    78054.55   
PJ corriente        467.96      661.69      752.15       729.4      975.68   

Participe            Maria  
Nacimiento      11/09/1995  
Alta            08/08/2018  
Salario 2021         35612  
Edad actuarial        26.0  
Edad entrada          23.0  
Ju

In [57]:
prestacion_definida(datos)

Hipotesis
Interés técnico valoración: 0.22%
Revalorizaciòn Salario: 1.5%
Revalorización Pensión: 1.0%
Beta Pension: 0.15
Participe             Joan        Pere       Marta     Claudia        Marc  \
Nacimiento      06/07/1973  03/12/1981  06/03/1957  09/11/1989  10/07/1990   
Alta            15/02/1998  08/03/2019  01/09/1985  09/10/2015  01/10/2010   
Salario 2021         29944       37586       60172       36236       47755   
Edad actuarial        49.0        41.0        65.0        32.0        31.0   
Edad entrada          25.0        38.0        28.0        26.0        19.0   
Jubilacion            2038        2046        2022        2054        2055   
Proyectado        37436.95    52935.26     60172.0    58351.71    78054.55   
PJ corriente        467.96      661.69      752.15       729.4      975.68   

Participe            Maria  
Nacimiento      11/09/1995  
Alta            08/08/2018  
Salario 2021         35612  
Edad actuarial        26.0  
Edad entrada          23.0  
Ju

In [66]:
int(round(((pd.to_datetime(datos['Alta']) - pd.to_datetime(datos['Nacimiento']))/365.25)/np.timedelta64(1, 'D'))[0])

25

In [44]:
fec_val= []
for i in range(0,6):
    fec_val.append('31/12/2021')
fec_val = pd.Series(fec_val)
round(((pd.to_datetime(fec_val) - pd.to_datetime(datos['Nacimiento']))/365.25)/np.timedelta64(1, 'D'))

0    49.0
1    41.0
2    65.0
3    32.0
4    31.0
5    26.0
dtype: float64

In [46]:
round(p1,2).iloc[-1]

213369.12

In [6]:
np.round(d1['Salario 2021']*r1**(65-1-d1['Edad actuarial']), 2)

0    41513.41
dtype: float64

In [76]:
array1 = []
for i in range(0, 3):
    array1.append(int(l2[i].split('/')[2]))

In [77]:
array1

[1973, 1989, 1995]

In [78]:
array2 = []
for i in range(0, 3):
    array2.append(int(l3[i].split('/')[2]))

In [81]:
[x-y for(x, y) in zip(array2, array1)]

[19, 21, 24]

In [38]:
pd.to_datetime(s, infer_datetime_format=True) 

0   2000-03-11
1   2000-03-12
2   2000-03-13
dtype: datetime64[ns]

# Planes de pensiones

### Aportaciò definida

In [102]:
t1 = np.arange(0, 660)  ## mensilidad
interes = 1.0125  # Interes (1+interes)
interes2 = interes**(1/12)  ## fraccionamiento 1/h
interes3 = interes2**-t1
len(interes3)  # factor financiero

660

In [58]:
55*12

660

In [2]:
raw1 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/Participes_aportacion.csv'
data1 = pd.read_csv(raw1)
raw2 = 'https://raw.githubusercontent.com/Joevalencia/Planes_pensione_ALM/main/approximation_planex.csv'
data2 = pd.read_csv(raw2)
del data1['Unnamed: 0']
del data2['Unnamed: 0']

In [13]:
data2.head()

,Antoni,Pere,Marta,Claudia,Marc,Maria
0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0
1,999215.1,999430.1,998523.0,999589.3,999602.4,999677.0
2,998430.2,998860.1,997046.1,999178.7,999204.7,999353.9
3,997645.4,998290.2,995569.1,998768.0,998807.1,999030.9
4,996860.5,997720.2,994092.1,998357.4,998409.5,998707.9


In [59]:
data1.Antoni[65]/data1.Antoni[49]

0.9590366479652812

In [16]:
data2.Antoni[65*12]

929171.0

In [92]:
px = (ok1.iloc[65*12+1:]/ok1.iloc[65*12]).reset_index()  ## Matriz de px mensuales Participes
px.to_csv('px_mens_participes.csv')

890.99

In [ ]:
f = ['m', 'n']
for i in f:
    var = np.roll(np.round(b1['PJ corriente'][i]*1.01**np.floor(np.arange(0,660) / 12 ), 2),-6)
    var[-6:] = var[-7]
    var[0] = 0
    var = pd.Series(var)

In [61]:
var = np.roll(np.round(b1['PJ corriente'][5]*1.01**np.floor(np.arange(0,660) / 12 ), 2),-8)
var[-8:] = var[-9]
var[0] = 0
var_Maria = pd.Series(var)
var_Maria

0         0.00
1      1638.18
2      1638.18
3      1638.18
4      1654.56
        ...   
655    2803.60
656    2803.60
657    2803.60
658    2803.60
659    2803.60
Length: 660, dtype: float64

In [65]:
variation = pd.DataFrame({'Joan_var': var_Joan, 'Pere_var':var_Pere, 'Marta_var':var_Marta, 
                          'Claudia_var':var_Claudia, 'Marc_var':var_Marc,'Maria_var':var_Maria} )
variation

,Joan_var,Pere_var,Marta_var,Claudia_var,Marc_var,Maria_var
0,0.00,0.00,0.00,0.00,0.00,0.00
1,890.99,1323.46,1278.66,1480.15,1950.67,1638.18
2,890.99,1323.46,1278.66,1494.95,1950.67,1638.18
3,890.99,1323.46,1278.66,1494.95,1950.67,1638.18
4,890.99,1323.46,1278.66,1494.95,1950.67,1654.56
...,...,...,...,...,...,...
655,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60
656,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60
657,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60
658,1524.85,2242.56,2188.31,2533.14,3338.40,2803.60


In [61]:
px.head()

,index,Antoni,Pere,Marta,Claudia,Marc,Maria
0,781,0.999621,0.999683,0.999737,0.999876,0.999740,0.999893
1,782,0.999242,0.999365,0.999474,0.999753,0.999480,0.999785
2,783,0.998863,0.999048,0.999212,0.999629,0.999220,0.999678
3,784,0.998485,0.998731,0.998949,0.999505,0.998961,0.999571
4,785,0.998106,0.998414,0.998686,0.999382,0.998701,0.999463


In [122]:
joan1 = ok['Joan_var']*ok1['Joan']
pere1 = ok['Pere_var']*ok1['Pere']
marta1 = ok['Marta_var']*ok1['Marta']
claudia1 = ok['Claudia_var']*ok1['Claudia']
marc1 = ok['Marc_var']*ok1['Marc']
maria1 = ok['Maria_var']*ok1['Maria']
new_df = pd.DataFrame({'Joan_VAA':joan1*interes3, 'Pere_VAA':pere1*interes3, 'Marta_VAA':marta1*interes3,
                       'Claudia_VAA':claudia1*interes3, 'Marc_VAA':marc1*interes3, 'Maria_VAA':maria1*interes3})
round(sum(new_df['Joan_VAA']), 2)

282648.57

In [128]:
VAAs = new_df.sum().round(2)
type(VAAs)

pandas.core.series.Series

In [140]:
VAAs

0    282648.57
1    430714.89
2    427609.83
3    554081.56
4    658762.27
5    621209.96
dtype: float64

In [165]:
jubi2 = pd.to_numeric(b1['Nacimiento'].str.split('/', expand=True)[2])+65
jubi2

0    2038
1    2046
2    2022
3    2054
4    2055
5    2060
Name: 2, dtype: int64

In [174]:
data1['Antoni'][65]

929171.0

In [184]:
VAA21_joan = round(VAAs[0]*(1.0125**-(2037-2021+1))*(data1['Antoni'][65]/data1['Antoni'][48]), 2)
VAA21_pere = round(VAAs[1]*(1.0125**-(2046-2021+1))*(data1['Pere'][65]/data1['Pere'][40]), 2)
VAA21_marta = round(VAAs[2]*(1.0125**-(2022-2021+1))*(data1['Marta'][65]/data1['Marta'][65]), 2)
VAA21_claudia = round(VAAs[3]*(1.0125**-(2054-2021+1))*(data1['Claudia'][65]/data1['Claudia'][32]), 2)
VAA21_marc = round(VAAs[4]*(1.0125**-(2055-2021+1))*(data1['Marc'][65]/data1['Marc'][32]), 2)
VAA21_maria = round(VAAs[5]*(1.0125**-(2060-2021+1))*(data1['Maria'][65]/data1['Maria'][26]), 2)
vaas21 = pd.Series([VAA21_joan, VAA21_pere, VAA21_marta, VAA21_claudia, VAA21_marc, VAA21_maria])
vaas21

0    219198.07
1    299408.56
2    417116.74
3    356065.37
4    412023.74
5    371243.25
dtype: float64

In [71]:
minidf = pd.DataFrame({'int Antoni':interes3, 'anual Antoni ':var}).join(pd.Series(px['Antoni']
                                                                                         , name='actual Antoni'))#.fillna(0)
minidf
#minidf.to_csv( 'Antoni.csv')

,int Antoni,anual Antoni,actual Antoni
0,1.000000,0.00,0.999621
1,0.998965,890.99,0.999242
2,0.997932,890.99,0.998863
3,0.996899,890.99,0.998485
4,0.995868,890.99,0.998106
...,...,...,...
655,0.507599,1524.85,0.000000
656,0.507074,1524.85,0.000000
657,0.506550,1524.85,0.000000
658,0.506026,1524.85,0.000000


In [65]:
var[520]

1366.76

In [72]:
VAA = round(sum(np.array(minidf['anual Antoni '])*np.array(minidf['actual Antoni'])*np.array(minidf['int Antoni'])),2)  ##Valor actual Actuarial
VAA

282648.57

In [74]:
minidf['Producto'] = minidf['int Antoni']*minidf['anual Antoni ']*minidf['actual Antoni']
minidf.head()

,int Antoni,anual Antoni,actual Antoni,Producto
0,1.000000,0.00,0.999621,0.000000
1,0.998965,890.99,0.999242,889.393742
2,0.997932,890.99,0.998863,888.136576
3,0.996899,890.99,0.998485,886.881155
4,0.995868,890.99,0.998106,885.627381


In [82]:
sum(minidf['Producto'])*((1.0125)**-(2037-2021+1))*(data1['Antoni'][65]/data1['Antoni'][48])  ## VAA Antoni 31/12/2021

219198.06344037587

In [78]:
data1['Antoni'][65]/data1['Antoni'][48]

970041.2

In [84]:
vaa_ant98 = sum(minidf['Producto'])*((1.0125)**-(2037-1998+1))*(data1['Antoni'][65]/data1['Antoni'][25]) ## VAA Antoni 31/12/1998

In [115]:
(VAA*1.0125**-(65-np.arange(19, 66))*(data1['Antoni'][65]/data1['Antoni'][np.arange(19, 66)]))[46]/46  ## Coste normal CU

4638.459110872414

In [ ]:
((VAA*1.0125**-(65-np.arange(19, 66))*(data1['Antoni'][65]/data1['Antoni'][np.arange(19, 66)]))[46]/46)

In [100]:
#data1['Antoni'][65]/data1['Antoni'][np.arange(19, 47)]

In [14]:
VAA = round(sum(np.array(minidf['var'])*np.array(minidf['act'])*np.array(minidf['int'])),2)  ##Valor actual Actuarial
VAA

265476.78

In [21]:
round((VAA*(interes**-(2037-2021+1))*.9578675),2)  ##VAA de las obligaciones del Plan de Pensiones a 31/12/2021. 


205881.08

In [22]:
round((VAA*(interes**-(2037-1998+1))*0.9452273),2)  #VAA de las obligaciones a 1992

152672.75

In [167]:
round(sum(np.array(minidf['var'])*np.array(minidf['act'])*np.array(minidf['int'])),2)  ##Valor actual Actuarial

265476.78

In [137]:
var = np.roll(np.round(864.85*1.01**np.floor(np.arange(0,673) / 12 ), 2),-6)
var[-6:] = var[-7]
var[0] = 0
var = pd.Series(var)
var.tail(8)

665    1494.91
666    1509.86
667    1509.86
668    1509.86
669    1509.86
670    1509.86
671    1509.86
672    1509.86
dtype: float64

In [61]:
int('06/07'.split('/')[1])  ## split meses

7

In [64]:
len(var)

607